In [ ]:
# === Imports === #
import MetaTrader5 as mt5
import pandas as pd
import numpy as np
import yaml
import os
from google_sheet_api import GoogleSheetsUploader

# === Setup Paths === #
BASE_DIR = os.getcwd()
CONFIG_PATH = os.path.join(BASE_DIR, "config.yaml")
CREDENTIAL_PATH = os.path.join(BASE_DIR, "credential_google_sheets.json")

# === Load Configuration === #
def load_config(path):
    """Load configuration from YAML file."""
    with open(path, "r") as file:
        return yaml.safe_load(file)

config = load_config(CONFIG_PATH)

# === Initialize MT5 Connection === #
if not mt5.initialize():
    print("MT5 initialization failed.")
    quit()

# === Retrieve Account Information === #
account_info = mt5.account_info()
if account_info is None:
    print("Failed to retrieve account information.")
    mt5.shutdown()
    quit()

# === Extract Balance, Equity, and Floating Loss === #
balance = account_info.balance
equity = account_info.equity
floating_loss = account_info.profit

# === Display Results === #
print("=== Account Information ===")
print(f"Balance: {balance:.2f} USD")
print(f"Equity: {equity:.2f} USD")
print(f"Floating Loss: {floating_loss:.2f} USD")

# === Fetch and Process Running Trades === #
positions = mt5.positions_get()
df_positions = pd.DataFrame([pos._asdict() for pos in positions]) if positions else pd.DataFrame()

if df_positions.empty:
    print("No running trades found. Error:", mt5.last_error())
else:
    # Filter and rename columns
    df_positions = df_positions[["type", "volume", "price_open", "price_current", "sl", "tp", "profit", "symbol"]]
    df_positions["type"] = df_positions["type"].replace({0: "Buy", 1: "Sell"})
    df_positions.rename(columns={
        "symbol": "Symbol",
        "type": "Action",
        "volume": "Lot",
        "price_open": "Price",
        "price_current": "Price Current",
        "sl": "SL",
        "tp": "TP",
        "profit": "Profit"
    }, inplace=True)

    # Aggregate by Symbol and Action
    def aggregate_group(group):
        return pd.Series({
            "Lot": group["Lot"].sum(),
            "Price": (group["Price"] * group["Lot"]).sum() / group["Lot"].sum(),
            "Price Current": group["Price Current"].mean(),
            "SL": group["SL"].mode().iloc[0] if not group["SL"].empty else np.nan,
            "TP": group["TP"].mode().iloc[0] if not group["TP"].empty else np.nan,
            "Profit": group["Profit"].sum(),
            "Warning": "SL Mismatch" if group["SL"].nunique() > 1 else np.nan
        })

    df_positions = (
        df_positions.groupby(["Symbol", "Action"], as_index=False)
        .apply(lambda group: aggregate_group(group))
        .reset_index(drop=True)
    )

# === Shutdown MT5 Connection === #
mt5.shutdown()

# === Calculate Coefficients === #
df_positions["Coeff"] = np.abs(df_positions["Profit"] / (df_positions["Price Current"] - df_positions["Price"]))

# === Update Coefficients in Config (Exclude NaN) === #
df_avg_coeff = df_positions.groupby("Symbol")["Coeff"].mean().reset_index()
differences = {}

for _, row in df_avg_coeff.iterrows():
    symbol = row["Symbol"]
    avg_coeff = round(row["Coeff"], 5)
    
    # Only update if coefficient is not NaN
    if not pd.isna(avg_coeff):
        if symbol in config.get("symbol_coefficients", {}):
            old_coeff = config["symbol_coefficients"].get(symbol, avg_coeff)
            if old_coeff != avg_coeff:
                differences[symbol] = {"Old": old_coeff, "New": avg_coeff}
        
        # Add or update the coefficient
        config["symbol_coefficients"][symbol] = avg_coeff

# === Save Updated Configuration === #
with open(CONFIG_PATH, "w") as file:
    yaml.safe_dump(config, file)

# === Display Changed Coefficients === #
print("Changed symbol_coefficients:")
for symbol, change in differences.items():
    print(f"{symbol}: {change['Old']} -> {change['New']}")

# Creating new rows for Balance, Equity, and Floating Loss
df_balance = pd.DataFrame([
    {"Symbol": "Balance", "Price": balance},
    {"Symbol": "Equity", "Price": equity},
    {"Symbol": "Floating Loss", "Price": floating_loss}
])

# Inserting the new rows at the top of the DataFrame
df_final = pd.concat([df_balance, df_positions], ignore_index=True)

# === Uploade Dataframe === #
uploader = GoogleSheetsUploader(CREDENTIAL_PATH, "Financial Report - Indonesia")
uploader.upload_dataframe(df_final, "Forex Summary", replace=True)

# === Display Final Merged DataFrame === #
print("Final Merged DataFrame:")
df_final

=== Account Information ===
Balance: 3731.05 USD
Equity: 3716.02 USD
Floating Loss: -1127.85 USD


C:\Users\syahr\AppData\Local\Temp\ipykernel_17472\1024628229.py:79: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_positions = df_positions_grouped.apply(lambda group: aggregate_group(group.loc[:, df_positions.columns.difference(["Symbol", "Action"])]))


Changed symbol_coefficients:
AAVEUSD: 0.5 -> 0.50147
ALGOUSD: 276.47059 -> 274.19355
APTUSD: 12.01011 -> 12.01028
ATOMUSD: 12.51572 -> 12.53247
AVAXUSD: 3.0031 -> 3.00325
BCHUSD: 0.10007 -> 0.10013
BNBUSD: 0.10007 -> 0.09992
DOTUSD: 12.52525 -> 12.55556
ETCUSD: 2.5 -> 2.51055
LRCUSD: 575.55556 -> 575.0
MANAUSD: 170.23256 -> 170.14218
NEARUSD: 25.03145 -> 25.0
SUIUSD: 30.01133 -> 30.05115
✅ Cleared all data from sheet: Forex Summary
✅ DataFrame successfully uploaded to Google Sheets: Forex Summary!
Final Merged DataFrame:


,Symbol,Price,Action,Lot,Price Current,SL,TP,Profit,Warning,Coeff
0,Balance,3731.050,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Equity,3716.020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Floating Loss,-1127.850,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AAPL.NAS,213.670,Buy,0.01,198.35,113.49,215.81,-15.32,NaN,1.000000
4,AAVEUSD,217.270,Buy,0.01,220.68,0.01,239.81,1.71,NaN,0.501466
...,...,...,...,...,...,...,...,...,...,...
174,WES.ASX,79.880,Buy,0.03,80.01,51.68,80.68,0.25,NaN,1.923077
175,WFC.NYSE,74.710,Buy,0.02,72.34,42.11,75.78,-4.74,NaN,2.000000
176,WMT.NYSE,98.820,Buy,0.03,96.58,72.42,99.89,-6.72,NaN,3.000000
177,XOM.NYSE,113.935,Buy,0.02,107.19,83.21,115.07,-13.49,NaN,2.000000
